# 機器學習程式碼

In [ ]:
# 整理機器學習特徵資料
# 2021/04/29 蘇彥庭
import json
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

from imblearn.over_sampling import RandomOverSampler 
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
import xgboost

import plotly.express as px
import seaborn as sns
from matplotlib import pyplot as plt
from pprint import pprint

import shap
shap.initjs()

import xgboost as xgb

# 設定pandas呈現表格時所有欄位都要呈現出來
pd.set_option('display.max_columns', None)

# 設定中文字體
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] 
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# 建立連線函數
def CreateDBEngine(secretFileName='dbSecret.json'):
    secretFile = json.load(open(secretFileName, 'r'))
    host = secretFile['host']
    username = secretFile['user']
    password = secretFile['password']
    port = secretFile['port']
    database = secretFile['dbName']
    return create_engine(f'mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}', echo=False)

# 整理SEGIS資料

In [ ]:
# 查詢內政部大數據鄉鎮市區資料指令
sqlQuery = '''
select
convert(replace(INFO_TIME, 'Y', ''), unsigned) as year,  # 年度
COUNTY_ID as county_id,  # 縣市代碼
COUNTY as county,  # 縣市名稱
TOWN_ID as town_id,  # 鄉鎮市區代碼
TOWN as town,  # 鄉鎮市區名稱

# 人口結構
sum(COLUMN2) as population,  # 人口數
sum(COLUMN52) as births_nums,  # 出生人口數
sum(COLUMN59) as deaths_nums,  # 死亡人口數
sum(COLUMN66) as immigration_nums,  # 遷入人口數
sum(COLUMN67) as migration_nums,  # 遷出人口數
round(sum(COLUMN52)/sum(COLUMN2)*1000, 4) as birth_rate,  # 粗出生率=出生人口數/人口數
round(sum(COLUMN59)/sum(COLUMN2)*1000, 4) as death_rate,  # 粗死亡率=死亡人口數/人口數
round((sum(COLUMN66)-sum(COLUMN68))/sum(COLUMN2)*1000, 4) as immigration_rate,  # 遷入率=(遷入人口數-鄉鎮區內住址變更之遷出人口數)/人口數
round((sum(COLUMN67)-sum(COLUMN69))/sum(COLUMN2)*1000, 4) as migration_rate,  # 遷出率=(遷出人口數-鄉鎮區內住址變更之遷出人口數)/人口數
round(sum(COLUMN52-COLUMN59)/sum(COLUMN2)*1000, 4) as nature_increase_rate,  # 自然增加率=(出生人口數-死亡人口數)/人口數
round(sum(COLUMN66-COLUMN67)/sum(COLUMN2)*1000, 4) as social_increase_rate,  # 社會增加率=(遷入人口數-遷出人口數)/人口數
round(sum(COLUMN33)/sum(COLUMN2)*100, 4) as toddler_population,  # 幼年人口佔比
round(sum(COLUMN34)/sum(COLUMN2)*100, 4) as worker_population,  # 青壯年人口佔比
round(sum(COLUMN35)/sum(COLUMN2)*100, 4) as elderly_population,  # 老年人口佔比
round(sum(COLUMN82)/sum(COLUMN2)*100, 4) as aboriginal_population_ratio,  # 原住民人口比率
round(sum(COLUMN33)/sum(COLUMN34)*100, 4) as child_dependency_ratio,  # 扶幼比
round(sum(COLUMN35)/sum(COLUMN34)*100, 4) as old_age_dependency_ratio,  # 扶老比

# 婚育概況
sum(COLUMN60) as married_pairs,  # 結婚對數
round(sum(COLUMN60)/sum(COLUMN2)*1000, 2) as married_rate,  # 粗結婚率
sum(COLUMN62) as divorce_pairs,  # 離婚對數
round(sum(COLUMN62)/sum(COLUMN2)*1000, 2) as divorce_rate,  # 粗離婚率(‰)(千分比)

# 托育教保
sum(COLUMN108+COLUMN109+COLUMN110) as age_0_2_population,  # 0-2歲學齡人口數
sum(COLUMN111+COLUMN112+COLUMN113+COLUMN114) as age_3_6_population  # 3-6學齡人口數

from project.demographic
group by info_time, county_id, county, town_id, town
order by COUNTY_ID, TOWN_ID, info_time;
'''

# 取得資料
segisData = pd.read_sql(sql=sqlQuery, con=CreateDBEngine())
segisData.head(10)

In [ ]:
# 為能夠對應其他來源資料 鄉鎮市區代碼若只有7碼 則第一碼補0
# 此為108年內政大數據資料應用組競賽用資料集_村里資料問題
reviseIdx = segisData['town_id'].str.len() == 7
segisData.loc[reviseIdx, 'town_id'] = '0' + segisData.loc[reviseIdx, 'town_id']

In [ ]:
# 判斷是否有缺值資料
segisData[segisData.isnull().any(axis=1)]

In [ ]:
# 針對缺值資料補0
segisData = segisData.fillna(0)

In [ ]:
# 確認是否有補齊
segisData[segisData.isnull().any(axis=1)]

In [ ]:
tempData = segisData[segisData['year'] == 108]
tempData.to_csv('segis_data.csv', encoding='utf-8-sig')

In [ ]:
# 建立特徵資料集
featureData = segisData[['year', 'town_id', 'county', 'town', 
                         'birth_rate',  # 出生率
                         'death_rate',  # 死亡率
                         'immigration_rate',  # 遷入率
                         'migration_rate',  # 遷出率
                         #'toddler_population',  # 幼年人口佔比
                         #'worker_population',  # 青壯年人口佔比
                         #'elderly_population',  # 老年人口佔比(拿掉避免共線性)
                         #'aboriginal_population_ratio',  # 原住民人口比率
                         'married_rate',  # 結婚率
                         'divorce_rate',  # 離婚率
                         'child_dependency_ratio',  # 扶幼比
                         'old_age_dependency_ratio'  # 扶老比
                        ]]
featureData

# 整理人口密度資料

In [ ]:
sqlQuery = '''
select convert(replace(info_time, 'Y', ''), unsigned) as year, county, town, sum(population)/sum(land_area) as density
from(
select info_time, county_id, county, town_id, town, 
COLUMN2 as population, COLUMN2/COLUMN7 as land_area from project.demographic) as t
group by info_time, county_id, county, town_id, town;
'''

# 取得資料
densityData = pd.read_sql(sql=sqlQuery, con=CreateDBEngine())
densityData

In [ ]:
# 判斷是否有缺值資料
densityData[densityData.isnull().any(axis=1)]

In [ ]:
# 將失業率併入特徵資料集
featureData = pd.merge(featureData,
                       densityData,
                       how='left',
                       on=['year', 'county', 'town'])
featureData.info()

# 整理可支配所得資料

In [ ]:
# 查詢可支配所得資料
sqlQuery = '''
select 
year,  # 年份
county,  # 縣市
disposable_income_median  # 家戶可支配所得中位數
from project.family_income_exp
where county != '總平均';
'''

# 取得資料
disposableIncomeData = pd.read_sql(sql=sqlQuery, con=CreateDBEngine())
disposableIncomeData

In [ ]:
# 判斷是否有缺值資料
disposableIncomeData[disposableIncomeData.isnull().any(axis=1)]

In [ ]:
# 將可支配所得中位數併入特徵資料集
featureData = pd.merge(featureData,
                       disposableIncomeData,
                       how='left',
                       on=['year', 'county'])
featureData.info()

# 整理不動產實價登錄資料

In [ ]:
# # 查詢不動產實價登錄資料
# sqlQuery = '''
# select 
# city as county,  # 縣市
# township as town,  # 鄉鎮市區
# convert(substr(transaction_day, 1, 3), unsigned) as year,
# total_price as price
# from project.real_price_immovables
# where transaction_sign in ('房地(土地+建物)', '房地(土地+建物)+車位');
# '''

# # 取得資料
# housePriceData = pd.read_sql(sql=sqlQuery, con=CreateDBEngine())
# housePriceData

In [ ]:
# # 校正資料
# housePriceData.loc[housePriceData['town']=='台東市', 'town'] = '臺東市'
# housePriceData.loc[housePriceData['town']=='屏東巿', 'town'] = '屏東市'  # 看起來一樣但實際不一樣
# housePriceData.loc[housePriceData['town']=='fa72埔鄉', 'town'] = '鹽埔鄉'
# housePriceData.loc[housePriceData['town']=='霧台鄉', 'town'] = '霧臺鄉'
# housePriceData.loc[housePriceData['town']=='頭份巿', 'town'] = '頭份市'  # 看起來一樣但實際不一樣
# housePriceData.loc[housePriceData['town']=='台西鄉', 'town'] = '臺西鄉'

In [ ]:
# # 判斷是否有缺值資料
# # housePriceData[housePriceData.isnull().any(axis=1)]
# housePriceData[housePriceData.drop('town', axis=1).isnull().any(axis=1)]

In [ ]:
# # 計算各年度鄉鎮市區房價中位數
# housePriceMedianData = housePriceData.groupby(by=['county', 'town', 'year'], as_index=False)['price'].\
#     agg(['median', 'count']).\
#     reset_index().\
#     rename(columns={'median': 'house_median_price', 'count': 'countNums'})

In [ ]:
# # 處理不動產實價登錄 新竹市及嘉義市未分區狀況 將縣市資料套用至鄉鎮市區
# for county in ['嘉義市', '新竹市']:
    
#     # 將不動產實價登錄的縣市資料取出 並丟棄town欄位
#     tempData = housePriceMedianData[housePriceMedianData['county'] == county].drop(columns=['town'])
#     # 將segis資料的鄉鎮市區取出 以cell方式存為town欄位
#     tempData['town'] = [segisData[segisData['county'] == county]['town'].unique().tolist()] * len(tempData)
#     # 利用explod將縣市資料套用至鄉鎮市區
#     tempData = tempData.explode('town')
#     # 刪除舊資料
#     housePriceMedianData = housePriceMedianData[housePriceMedianData['county'] != county]
#     # 併入新資料
#     housePriceMedianData = pd.concat([housePriceMedianData, tempData])

In [ ]:
# # 建立完整索引清單
# housePriceMedianData = pd.merge(segisData[['year', 'county', 'town']],
#                                 housePriceMedianData,
#                                 how='left',
#                                 on=['year', 'county', 'town'])

# # 併入可支配所得資料
# housePriceToIncome = pd.merge(housePriceMedianData,
#                               disposableIncomeData,
#                               how='left',
#                               on=['year', 'county'])

In [ ]:
# # 排序資料確保向前補值不會出錯
# housePriceToIncome = housePriceToIncome.sort_values(by=['county', 'town', 'year']).reset_index(drop=True)

# # 處理缺值房價中位數缺值問題 若有缺值則向前補足
# housePriceToIncome['house_median_price'] = housePriceToIncome. \
# groupby(['county', 'town'])['house_median_price']. \
# transform(lambda x: x.ffill())

# # 處理缺值房價中位數缺值問題 若有缺值則向前補足
# housePriceToIncome['disposable_income_median'] = housePriceToIncome. \
# groupby(['county', 'town'])['disposable_income_median']. \
# transform(lambda x: x.ffill())

# housePriceToIncome

In [ ]:
# 確認資料
# housePriceToIncome[housePriceToIncome['town']=='屏東市']

In [ ]:
# # 計算房價所得比
# housePriceToIncome['location_house_price_to_income'] = housePriceToIncome['house_median_price']/\
#                                               housePriceToIncome['disposable_income_median']
# housePriceToIncome

In [ ]:
# # 判斷是否有缺值資料
# # housePriceToIncome[housePriceToIncome.isnull().any(axis=1)]

# checkData = housePriceToIncome[(~housePriceToIncome['county'].isin(['金門縣', '連江縣'])) &
#                                (~housePriceToIncome['year'].isin([104, 109]))]
# checkData[checkData.drop('countNums', axis=1).isnull().any(axis=1)]

In [ ]:
# # 將可支配所得中位數與房價所得比併入特徵資料集
# featureData = pd.merge(featureData,
#                        housePriceToIncome[['year', 'county', 'town', 
#                                            'disposable_income_median', 
#                                            'location_house_price_to_income']],
#                        how='left',
#                        on=['year', 'county', 'town'])
# featureData.info()

# 整理失業率資料

In [ ]:
# 查詢失業率資料
sqlQuery = '''
select year, area as county, total as unemployment_rate from project.unemployment_rate;
'''

# 取得資料
unemploymentRateData = pd.read_sql(sql=sqlQuery, con=CreateDBEngine())
unemploymentRateData

In [ ]:
# 將失業率併入特徵資料集
featureData = pd.merge(featureData,
                       unemploymentRateData,
                       how='left',
                       on=['year', 'county'])
featureData.info()

In [ ]:
# 判斷失業率缺值資料狀況
# 連江縣和金門縣沒有資料
featureData[featureData[['year', 'county', 'unemployment_rate']].isnull().any(axis=1)]

# 整理房價負擔能力統計(縣市)

In [ ]:
# 查詢房價負擔能力統計資料
sqlQuery = '''
select year, 
city as county, 
mbr as house_burden_ratio,  # 房價負擔率(%)
round(year_change_mbr, 2) as house_burden_yoy,   # 房價負擔率年變動值
pir as house_price_income_ratio,  # 房價所得比(倍)
round(year_change_pir, 2) as house_price_income_yoy  # 房價所得比年變動值
from project.house_burden_data where season = 4;
'''

# 取得資料
housingAffordabilityData = pd.read_sql(sql=sqlQuery, con=CreateDBEngine())
housingAffordabilityData

In [ ]:
# 將房價負擔能力統計資料併入特徵資料集
featureData = pd.merge(featureData,
                       housingAffordabilityData,
                       how='left',
                       on=['year', 'county'])
featureData.info()

# 整理女性勞動力參與率

In [ ]:
sqlQuery = '''
select year, 
area as county,
age_20_24_female as female_labor_ratio_20_24,  # 20-24歲女性勞動力參與率(%)
age_25_29_female as female_labor_ratio_25_29,  # 25-29歲女性勞動力參與率(%)
age_30_34_female as female_labor_ratio_30_34,  # 30-34歲女性勞動力參與率(%)
age_35_39_female as female_labor_ratio_35_39  # 35-39歲女性勞動力參與率(%)
from project.labor_force_participation_rate;
'''

# 取得資料
laborForceData = pd.read_sql(sql=sqlQuery, con=CreateDBEngine())
laborForceData

In [ ]:
# 判斷是否有缺值資料
laborForceData[laborForceData.isnull().any(axis=1)]

In [ ]:
# 將女性勞動力參與率統計資料併入特徵資料集
featureData = pd.merge(featureData,
                       laborForceData,
                       how='left',
                       on=['year', 'county'])
featureData.info()

# 整理自有住宅率

In [ ]:
sqlQuery = '''
select year, 
county as county, 
home_ownership_rate as home_ownership_rate  # 自有住宅率(%)
from project.home_ownership_rate;
'''

# 取得資料
homeOwnershipData = pd.read_sql(sql=sqlQuery, con=CreateDBEngine())
homeOwnershipData

In [ ]:
# 將自有住宅率資料併入特徵資料集
featureData = pd.merge(featureData,
                       homeOwnershipData,
                       how='left',
                       on=['year', 'county'])
featureData.info()

# 幼兒園概況資訊

In [ ]:
sqlQuery = '''
select year, county,
round(public_school_nums/total_school_nums, 4) as public_kindergarten_ratio,  # 公幼占比
round(public_children/total_children, 4) as public_children_ratio  # 公幼生占比
from project.kindergarten_data;
'''

# 取得資料
kindergartenData = pd.read_sql(sql=sqlQuery, con=CreateDBEngine())
kindergartenData

In [ ]:
# 將幼兒(稚)園概況統計資料併入特徵資料集
featureData = pd.merge(featureData,
                       kindergartenData,
                       how='left',
                       on=['year', 'county'])
featureData.info()

#  整理育齡婦女之年齡別生育率

In [ ]:
# sqlQuery = '''
# select year, county,
# age20_24_childbearing,  # 20-24歲生育率
# age25_29_childbearing,  # 25-29歲生育率
# age30_34_childbearing,  # 30-34歲生育率
# age35_39_childbearing,  # 35-39歲生育率
# mean_age_childbearing,  # 生母生育平均年齡
# mean_age_first_childbearing  # 生母生第一胎平均年齡
# from project.fertility_rate;
# '''

# # 取得資料
# fertilityRatioData = pd.read_sql(sql=sqlQuery, con=CreateDBEngine())
# fertilityRatioData

In [ ]:
# # 將育齡婦女之年齡別生育率統計資料併入特徵資料集
# featureData = pd.merge(featureData,
#                        fertilityRatioData,
#                        how='left',
#                        on=['year', 'county'])
# featureData.info()

# 整理公辦民營托嬰中心資料

In [ ]:
sqlQuery = '''
select year-1911 as year,
city as county,
# 公辦民營托嬰中心占比 = 公辦民營托嬰中心所數/托嬰中心所數
round(public_to_private_childcare_center/totla_childcare_center, 4) as public_child_care_center_ratio
from project.number_of_childcare_centers_and_recipients;
'''

# 取得資料
childcareCentersData = pd.read_sql(sql=sqlQuery, con=CreateDBEngine())
childcareCentersData

In [ ]:
# 將公辦民營托嬰中心統計資料併入特徵資料集
featureData = pd.merge(featureData,
                       childcareCentersData,
                       how='left',
                       on=['year', 'county'])
featureData.info()

In [ ]:
0

In [ ]:
# 將托嬰涵蓋比率資料併入特徵資料集
featureData = pd.merge(featureData,
                       childCareCoverRatio,
                       how='left',
                       on=['year', 'county'])
featureData.info()

# 儲存特徵資料

In [ ]:
# 排除金門縣及馬祖縣(政府資料不足)
featureData = featureData[~featureData['county'].isin(['金門縣', '連江縣'])]

In [ ]:
# 確認特徵資料狀況
featureData.info()

In [ ]:
# 確認缺值欄位
checkData = featureData[featureData['year']!=109]
checkData[checkData.isnull().any(axis=1)]

In [ ]:
# 儲存資料
featureData.to_csv('featureData.csv', index=False, encoding='utf-8-sig')

# 機器學習前置作業

In [ ]:
# 讀取資料
featureData = pd.read_csv('featureData.csv')

In [ ]:
# # 產生預測目標: 以t+1年的出生率作為預測目標
# featureData['predictY'] = featureData.sort_values(by=['town_id', 'year'], ascending=[True, True]).groupby(['town_id'])['birth_rate'].shift(-1)
# featureData

In [ ]:
# # 產生預測目標分類: 出生率是否增加/下降 t+1年的出生率>t年出生率視為增加 反之為下降
# featureData['predictYClass'] = np.where(featureData['predictY'] > featureData['birth_rate'], 1, 0)
# featureData

In [ ]:
# 產生預測目標: 以t年的出生率作為預測目標
featureData['predictY'] = featureData['birth_rate']
featureData

In [ ]:
# # 產生預測目標分類: 出生率是否增加/下降 t年的出生率>t-1年出生率視為增加 反之為下降
featureData['lag_birth_rate'] = featureData.sort_values(by=['town_id', 'year'], ascending=[True, True]).groupby(['town_id'])['birth_rate'].shift(1)
featureData['predictYClass'] = np.where(featureData['birth_rate'] > featureData['lag_birth_rate'], 1, 0)
featureData

In [ ]:
# 劃分訓練期資料(以104年-107年作為訓練樣本)
# trainData = featureData[featureData['year'].isin(list(range(104, 108)))].reset_index(drop=True)
# 劃分訓練期資料(以105年-107年作為訓練樣本)
trainData = featureData[featureData['year'].isin(list(range(105, 108)))].reset_index(drop=True)

# 刪除遺失值
trainData = trainData.dropna()

# 分離樣本識別資訊與機器學習用訓練資料
trainDataInfo = trainData[['year', 'town_id', 'county', 'town']]

# 訓練預測目標
y_train_reg = trainData['predictY']
y_train_class = trainData['predictYClass']

# 訓練特徵資料
X_train_raw = trainData.drop(['year', 'town_id', 'county', 'town', 'predictY',
                              'predictYClass', 'birth_rate', 'lag_birth_rate'], axis=1)

In [ ]:
# 建立特徵中文名稱
chColumnNames = {'death_rate': '死亡率',
                 'immigration_rate': '移入率',
                 'migration_rate': '移出率',
                 'toddler_population': '幼年人口佔比',
                 'worker_population': '青壯年人口佔比',
                 'elderly_population': '老年人口佔比',
                 'child_dependency_ratio': '扶幼比',
                 'old_age_dependency_ratio': '扶老比',
                 'aboriginal_population_ratio': '原住民人口比率',
                 'married_rate': '結婚率',
                 'divorce_rate': '離婚率',
                 'density': '人口密度',
                 'disposable_income_median': '家戶可支配所得中位數',
                 'location_house_price_to_income': '房價所得比(鄉鎮市區)',
                 'unemployment_rate': '失業率',
                 'house_burden_ratio': '房價負擔率',
                 'house_burden_yoy': '房價負擔率YoY',
                 'house_price_income_ratio': '房價所得比',
                 'house_price_income_yoy': '房價所得比YoY',
                 'female_labor_ratio_20_24': '20-24歲女性勞動力參與率',
                 'female_labor_ratio_25_29': '25-29歲女性勞動力參與率',
                 'female_labor_ratio_30_34': '30-34歲女性勞動力參與率',
                 'female_labor_ratio_35_39': '35-39歲女性勞動力參與率',
                 'home_ownership_rate': '自有住宅率',
                 'public_kindergarten_ratio': '公幼佔比',
                 'public_children_ratio': '公幼生佔比',
                 'age20_24_childbearing': '20-24歲生育率',
                 'age25_29_childbearing': '25-29歲生育率',
                 'age30_34_childbearing': '30-34歲生育率',
                 'age35_39_childbearing': '35-39歲生育率',
                 'mean_age_childbearing': '生母生育平均年齡',
                 'mean_age_first_childbearing': '生母生第一胎平均年齡',
                 'public_child_care_center_ratio': '公辦民營托嬰中心占比',
                 'child_care_cover_ratio': '托嬰涵蓋比率'
                }

featureChName = [chColumnNames[elem] for elem in X_train_raw.columns]
featureChName

In [ ]:
# 檢查資料年份
trainData['year'].drop_duplicates()

In [ ]:
# 計算訓練期相關係數矩陣
trainCorrMatrix = trainData.drop(['year', 'town_id', 'county', 'town', 'predictYClass', 
                                  'birth_rate', 'lag_birth_rate'], axis=1).corr()
trainCorrMatrix.style.background_gradient(cmap='coolwarm')

In [ ]:
# 繪製t期各變數與t+1期相關係數值
fig = px.bar(trainCorrMatrix['predictY'])
fig.update_layout(title={'text': 't+1期出生率與t期特徵之相關係數', 'x': 0.5},
                  showlegend=False,
                  xaxis_title="特徵名稱",
                  yaxis_title="相關係數")
fig.show()

In [ ]:
# 檢驗共線性
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif_data = pd.DataFrame()
vif_data["feature"] = X_train_raw.columns
vif_data["VIF"] = [variance_inflation_factor(X_train_raw.values, i) for i in range(len(X_train_raw.columns))]
vif_data[vif_data["VIF"]>10]

In [ ]:
# # 觀察每個變數的分布
# plotData = featureData
# print(plotData.columns)
# plotData['資料類別'] = np.where(plotData['year'].isin(list(range(104, 108))), '訓練集', '測試集')
# fig = px.histogram(plotData, x="birth_rate", color = "資料類別")
# fig.show()

In [ ]:
# # 觀察每個變數的分布
# plotData
# sns.pairplot(data = plotData, vars=['predictY','priceIncomeRatio_yoy','unemployment_rate'])

In [ ]:
# 劃分測試期資料(以108年作為測試樣本)
testData = featureData[featureData['year'].isin([108])].reset_index(drop=True)

# 刪除遺失值
testData = testData.dropna()

# 分離樣本識別資訊與機器學習用訓練資料
testDataInfo = testData[['year', 'town_id', 'county', 'town']]

# 測試預測目標
y_test_reg = testData['predictY']
y_test_class = testData['predictYClass']

# 測試特徵資料
X_test_raw = testData.drop(['year', 'town_id', 'county', 'town', 'predictY',
                            'predictYClass', 'birth_rate', 'lag_birth_rate'], axis=1)

In [ ]:
# 檢查資料年份
testData['year'].drop_duplicates()

In [ ]:
# 觀察樣本狀況
print('訓練集樣本共有:', len(X_train_raw), '筆樣本')
print('訓練集樣本共有:', len(X_test_raw), '筆樣本')

In [ ]:
# 繪製訓練集與測試集分類數狀況
plotData = pd.concat([y_train_class.value_counts().reset_index().assign(dataType='訓練集'),
                      y_test_class.value_counts().reset_index().assign(dataType='測試集')])
plotData.columns = ['預測分類', '樣本數量', '資料類別']
plotData['預測分類'] = plotData['預測分類'].astype('str')
fig = px.bar(plotData, x='資料類別', y='樣本數量', color='預測分類', barmode='group')
fig.show()

In [ ]:
# 資料標準化(z-score)
scaler  = preprocessing.StandardScaler().fit(X_train_raw)
X_train_scaler = scaler.transform(X_train_raw)
X_test_scaler = scaler.transform(X_test_raw)

# 觀察轉換狀況
pd.DataFrame(X_train_scaler).hist()
plt.show()

In [ ]:
# 處理極值
X_train_scaler[X_train_scaler>3] = 3
X_train_scaler[X_train_scaler<-3] = -3

# 確認矩陣值域
print('特徵矩陣最大值:', np.amax(X_train_scaler))
print('特徵矩陣最小值:', np.amin(X_train_scaler))

In [ ]:
# 資料標準化(縮放至[0, 1]區間)
min_max_scaler = preprocessing.MinMaxScaler().fit(X_train_scaler)
X_train_minmax = min_max_scaler.transform(X_train_scaler)
X_test_minmax = min_max_scaler.transform(X_test_scaler)

# 確認觀察矩陣值域
print('特徵矩陣最大值:', np.amax(X_train_minmax))
print('特徵矩陣最小值:', np.amin(X_train_minmax))

In [ ]:
# 特徵標準化後的資料
X_train = X_train_minmax
X_test = X_test_minmax

In [ ]:
# 不做特徵標準化
X_train = X_train_raw
X_test = X_test_raw

In [ ]:
# 由於訓練集資料不平衡 此處採用Oversample
ros = RandomOverSampler(random_state=666)
X_train_class, y_train_class = ros.fit_resample(X_train, y_train_class)

In [ ]:
# 確認訓練集是否已達成oversample
plotData = pd.concat([y_train_class.value_counts().reset_index().assign(dataType='訓練集'),
                      y_test_class.value_counts().reset_index().assign(dataType='測試集')])
plotData.columns = ['預測分類', '樣本數量', '資料類別']
plotData['預測分類'] = plotData['預測分類'].astype('str')
fig = px.bar(plotData, x='資料類別', y='樣本數量', color='預測分類', barmode='group')
fig.show()

# 多元線性迴歸(預測值)

In [ ]:
from sklearn import linear_model

# 建立線性迴歸模型
regModel = linear_model.LinearRegression()

# 學習線性迴歸模型
regModel = regModel.fit(X_train, y_train_reg)

# 迴歸模型預測
reg_train_pred = regModel.predict(X_train)
reg_test_pred = regModel.predict(X_test)

In [ ]:
print('迴歸模型預測train的RMSE:', mean_squared_error(y_train_reg, reg_train_pred, squared=False))
print('迴歸模型預測test的RMSE:', mean_squared_error(y_test_reg, reg_test_pred, squared=False))

# 決策樹(預測值)

In [ ]:
from sklearn import tree

# 建立決策樹模型
decisionTreeRegModel = tree.DecisionTreeRegressor()

# 查看 Decision Tree 默認參數值
pprint(decisionTreeRegModel.get_params())

# 設定參數組合
paramGrid = {
    'min_samples_leaf': [1, 3, 5, 10],  # 葉子節點最少樣本數
    'max_depth': [3, 5, 7, 10],  # 最大深度
}

# 設定 Grid Search with Cross Validation
grid_search = GridSearchCV(estimator = decisionTreeRegModel, param_grid = paramGrid, 
                           cv = 5, n_jobs = -1, verbose = 2)

# 執行 Grid Search with Cross Validation
grid_search.fit(X_train, y_train_reg)

In [ ]:
# 最佳參數組合
print('最佳參數組合:')
pprint(grid_search.best_params_)
print()

# 最佳參數組合訓練的 Decision Tree 模型
bestDecisionTreeRegModel = grid_search.best_estimator_

# 模型預測
bestDecisionTreeReg_train_pred = bestDecisionTreeRegModel.predict(X_train)
bestDecisionTreeReg_test_pred = bestDecisionTreeRegModel.predict(X_test)

print('最佳參數組合決策樹迴歸模型預測train的RMSE:', 
      mean_squared_error(y_train_reg, bestDecisionTreeReg_train_pred, squared=False))
print('最佳參數組合決策樹迴歸模型預測test的RMSE:', 
      mean_squared_error(y_test_reg, bestDecisionTreeReg_test_pred, squared=False))

In [ ]:
# # 繪製決策樹
# tree.plot_tree(bestDecisionTreeRegModel)
# plt.show()

# 隨機森林迴歸(預測值)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# 建立隨機森林模型
rfRegModel = RandomForestRegressor()

# 查看 Random Forest默認參數值
# pprint(rfRegModel.get_params())

# 設定參數組合
paramGrid = {
    'min_samples_leaf': [1, 3, 5, 10],  # 葉子節點最少樣本數
    'max_depth': [3, 5, 7, 10],  # 最大深度
    'n_estimators': [100, 200, 300, 500]  # 決策樹數量
}

# 設定 Grid Search with Cross Validation
grid_search = GridSearchCV(estimator = rfRegModel, param_grid = paramGrid, 
                           cv = 5, n_jobs = -1, verbose = 2)

# 執行 Grid Search with Cross Validation
grid_search.fit(X_train, y_train_reg)

In [ ]:
# 最佳參數組合
print('最佳參數組合:')
pprint(grid_search.best_params_)
print()

# 最佳參數組合訓練的隨機森林模型
bestRfRegModel = grid_search.best_estimator_

# 模型預測
rfReg_train_pred = bestRfRegModel.predict(X_train)
rfReg_test_pred = bestRfRegModel.predict(X_test)

print('最佳參數組合隨機森林迴歸模型預測train的RMSE:', mean_squared_error(y_train_reg, rfReg_train_pred, squared=False))
print('最佳參數組合隨機森林迴歸模型預測test的RMSE:', mean_squared_error(y_test_reg, rfReg_test_pred, squared=False))

In [ ]:
# 繪製特徵解釋能力
importance = bestRfRegModel.feature_importances_
plotData = pd.DataFrame({'特徵名稱': X_train_raw.columns, '特徵重要程度': importance})
fig = px.bar(plotData, x='特徵名稱', y='特徵重要程度')
fig.show()

In [ ]:
# 繪製SHAP圖形
shap_values = shap.TreeExplainer(bestRfRegModel).shap_values(X_train)
shap.summary_plot(shap_values, X_train, feature_names=featureChName)

# SVR(預測值)

In [ ]:
from sklearn import svm

# 建立SVR模型
svrModel = svm.SVR()

# # 查看 SVR 默認參數值
# pprint(svrModel.get_params())

# 設定參數組合
paramGrid = {
    'C': [2**i for i in range(-10, 11, 2)],  # cost值
    'gamma': [2**i for i in range(-10, 11, 2)],  # gamma值
}

# 設定 Grid Search with Cross Validation
grid_search = GridSearchCV(estimator = svrModel, param_grid = paramGrid, 
                           cv = 5, n_jobs = -1, verbose = 2)

# 執行 Grid Search with Cross Validation
grid_search.fit(X_train, y_train_reg)

In [ ]:
# 最佳參數組合
print('最佳參數組合:')
pprint(grid_search.best_params_)
print()

# 最佳參數組合訓練的SVR模型
bestSvrModel = grid_search.best_estimator_

# 模型預測
svr_train_pred = bestSvrModel.predict(X_train)
svr_test_pred = bestSvrModel.predict(X_test)

print('最佳參數組合SVR模型預測train的RMSE:', mean_squared_error(y_train_reg, svr_train_pred, squared=False))
print('最佳參數組合SVR模型預測test的RMSE:', mean_squared_error(y_test_reg, svr_test_pred, squared=False))

# XGBoost(預測值)

In [ ]:
# 建立XGBoost Data Matrix
trainDMatrix = xgb.DMatrix(data=X_train, label=y_train_reg, feature_names=featureChName)
testDMatrix = xgb.DMatrix(data=X_test, label=y_test_reg, feature_names=featureChName)

In [ ]:
from sklearn.model_selection import ParameterGrid

# 設定參數組合
paramGrid = {
    'objective':['reg:squarederror'],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5, 7, 10],
    'colsample_bytree': [0.7, 0.9],
    'subsample': [0.7, 0.9]
}
paramGrid = list(ParameterGrid(paramGrid))

In [ ]:
# 迴圈計算最佳參數組合
cvTestEval = list()
cvTestRounds = list()
for params in paramGrid:
    
    # 執行交叉驗證
    cv_results = xgb.cv(params=params,
                        dtrain=trainDMatrix,
                        num_boost_round=1000,
                        nfold=5,
                        metrics="rmse",
                        early_stopping_rounds=20)
    
    
    # 紀錄交叉驗證平均誤差
    cvTestEval.append(float(cv_results['test-rmse-mean'].tail(1)))
    
    # 紀錄XGBoost最適疊代次數
    cvTestRounds.append(len(cv_results))

In [ ]:
# 取test-rmse-mean最小值的參數組合
bestParamsIdx = cvTestEval.index(min(cvTestEval))
bestParams = paramGrid[bestParamsIdx]
bestRounds = cvTestRounds[bestParamsIdx]
print('XGBoost最佳參數組合:', bestParams)
print('XGBoost最佳疊代數:', bestRounds)

In [ ]:
# 以最佳參數組合訓練模型
xgbModel = xgb.train(params=bestParams, dtrain=trainDMatrix, num_boost_round=bestRounds)

In [ ]:
# 模型預測
xgboost_train_pred = xgbModel.predict(trainDMatrix)
xgboost_test_pred = xgbModel.predict(testDMatrix)

print('最佳參數組合XGBoost模型預測train的RMSE:', mean_squared_error(y_train_reg, xgboost_train_pred, squared=False))
print('最佳參數組合XGBoost模型預測test的RMSE:', mean_squared_error(y_test_reg, xgboost_test_pred, squared=False))

In [ ]:
xgboost.plot_importance(xgbModel, importance_type='gain')

In [ ]:
# 進行SHAP分析
explainer = shap.TreeExplainer(xgbModel)
shap_values = explainer.shap_values(trainDMatrix)

In [ ]:
shap.summary_plot(shap_values, X_train, feature_names=featureChName)

In [ ]:
# 分析單筆樣本
shap_values = explainer(X_test)
ind = 201
print("預測鄉鎮市區:", testDataInfo["county"][ind]+testDataInfo["town"][ind])
print("預測值:", xgboost_test_pred[ind])
print("實際值:", y_test_reg[ind])
shap.plots.force(shap_values[ind], feature_names=featureChName)

In [ ]:
# 整理預測結果
testOutput = testDataInfo.copy()
testOutput.insert(loc=len(testOutput.columns), column='realY', value=y_test_reg)
testOutput.insert(loc=len(testOutput.columns), column='predictY', value=xgboost_test_pred)
testOutput

# XGBoost(預測分類)

In [ ]:
# 建立XGBoost Data Matrix
trainDMatrix = xgb.DMatrix(data=X_train_class, label=y_train_class, feature_names=featureChName)
testDMatrix = xgb.DMatrix(data=X_test, label=y_test_class, feature_names=featureChName)

In [ ]:
from sklearn.model_selection import ParameterGrid

# 設定參數組合
paramGrid = {
    'objective':['binary:logistic'],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5, 7, 10],
    'colsample_bytree': [0.7, 0.9],
    'subsample': [0.7, 0.9]
}
paramGrid = list(ParameterGrid(paramGrid))

In [ ]:
# 迴圈計算最佳參數組合
cvTestEval = list()
cvTestRounds = list()
for params in paramGrid:
    
    # 執行交叉驗證
    cv_results = xgb.cv(params=params,
                        dtrain=trainDMatrix,
                        num_boost_round=1000,
                        nfold=5,
                        metrics="logloss",
                        early_stopping_rounds=20)
    
    
    # 紀錄交叉驗證平均誤差
    cvTestEval.append(float(cv_results['test-logloss-mean'].tail(1)))
    
    # 紀錄XGBoost最適疊代次數
    cvTestRounds.append(len(cv_results))

In [ ]:
# 取test-logloss-mean最小值的參數組合
bestParamsIdx = cvTestEval.index(min(cvTestEval))
bestParams = paramGrid[bestParamsIdx]
bestRounds = cvTestRounds[bestParamsIdx]
print('XGBoost最佳參數組合:', bestParams)
print('XGBoost最佳疊代數:', bestRounds)

In [ ]:
# 以最佳參數組合訓練模型
xgbModel = xgb.train(params=bestParams, dtrain=trainDMatrix, num_boost_round=bestRounds)

In [ ]:
# 模型預測
xgboost_train_pred = xgbModel.predict(trainDMatrix)
xgboost_train_pred = np.where(xgboost_train_pred > 0.5, 1, 0)
xgboost_test_pred = xgbModel.predict(testDMatrix)
xgboost_test_pred = np.where(xgboost_test_pred > 0.5, 1, 0)

print('最佳參數組合XGBoost模型預測train的accuracy:', accuracy_score(y_train_class, xgboost_train_pred))
print('最佳參數組合XGBoost模型預測test的accuracy:', accuracy_score(y_test_class, xgboost_test_pred))

# logistic迴歸(預測分類)

In [ ]:
from sklearn.linear_model import LogisticRegression

# 建立logistic迴歸模型
logisticRegModel = LogisticRegression()

# # 查看logistic迴歸默認參數值
# pprint(logisticRegModel.get_params())

# 訓練logistic迴歸模型
logisticRegModel.fit(X_train_class, y_train_class)

# 模型預測
logistic_train_pred = logisticRegModel.predict(X_train_class)
logistic_test_pred = logisticRegModel.predict(X_test)

print('最佳參數組合logistic模型預測train的accuracy:', accuracy_score(y_train_class, logistic_train_pred))
print('最佳參數組合logistic模型預測test的accuracy:', accuracy_score(y_test_class, logistic_test_pred))

In [ ]:
# # 模型儲存與載入
# # 模型儲存
# filename = 'logisticRegModel.sav'
# pickle.dump(logisticRegModel, open(filename, 'wb'))

# # 讀取模型
# logisticRegModel = pickle.load(open(filename, 'rb'))

# # 模型預測
# logistic_train_pred = logisticRegModel.predict(X_train_class)
# logistic_test_pred = logisticRegModel.predict(X_test) 

In [ ]:
# SHAP分析
# https://shap.readthedocs.io/en/latest/example_notebooks/tabular_examples/linear_models/Sentiment%20Analysis%20with%20Logistic%20Regression.html
explainer = shap.Explainer(logisticRegModel, X_train_class, feature_names=featureChName)
shap_values = explainer(X_train_class)

In [ ]:
shap.plots.beeswarm(shap_values, max_display=30)

In [ ]:
# 儲存SHAP圖形輸出
shap.plots.beeswarm(shap_values, max_display=30, show=False)
plt.savefig('shap.png')

In [ ]:
# 觀察模型如何預測
shap_values = explainer(X_test)
shap.plots.beeswarm(shap_values, max_display=30)

In [ ]:
ind = 307
print("預測鄉鎮市區:", testDataInfo["county"][ind]+testDataInfo["town"][ind])
print("預測結果:", "出生率增加" if logistic_test_pred[ind] else "出生率下降")
print("實際結果:", "出生率增加" if y_test_class[ind] else "出生率下降")
shap.plots.force(shap_values[ind])

# 決策樹(預測分類)

In [ ]:
from sklearn import tree

# 建立決策樹模型
decisionTreeClassModel = tree.DecisionTreeClassifier()

# # 查看 Decision Tree 默認參數值
# pprint(decisionTreeClassModel.get_params())

# 設定參數組合
paramGrid = {
    'min_samples_leaf': [1, 3, 5, 10],  # 葉子節點最少樣本數
    'max_depth': [3, 5, 7, 10],  # 最大深度
}

# 設定 Grid Search with Cross Validation
grid_search = GridSearchCV(estimator = decisionTreeClassModel, param_grid = paramGrid, 
                           cv = 5, n_jobs = -1, verbose = 2)

# 執行 Grid Search with Cross Validation
grid_search.fit(X_train_class, y_train_class)

In [ ]:
# 最佳參數組合
print('最佳參數組合:')
pprint(grid_search.best_params_)
print()

# 最佳參數組合訓練的決策樹模型
bestDecisionTreeClassModel = grid_search.best_estimator_

# 模型預測
decisionTreeClass_train_pred = bestDecisionTreeClassModel.predict(X_train_class)
decisionTreeClass_test_pred = bestDecisionTreeClassModel.predict(X_test)

print('最佳參數組合決策樹分類模型預測train的accuracy:', accuracy_score(y_train_class, decisionTreeClass_train_pred))
print('最佳參數組合決策樹分類模型預測test的accuracy:', accuracy_score(y_test_class, decisionTreeClass_test_pred))

# 隨機森林分類(預測分類)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# 建立隨機森林模型
rfClassModel = RandomForestClassifier()

# # 查看 Random Forest默認參數值
# pprint(rfClassModel.get_params())

# 設定參數組合
paramGrid = {
    'min_samples_leaf': [1, 3, 5, 10],  # 葉子節點最少樣本數
    'max_depth': [3, 5, 7, 10],  # 最大深度
    'n_estimators': [100, 200, 300, 500]  # 決策樹數量
}

# 設定 Grid Search with Cross Validation
grid_search = GridSearchCV(estimator = rfClassModel, param_grid = paramGrid, 
                           cv = 5, n_jobs = -1, verbose = 2)

# 執行 Grid Search with Cross Validation
grid_search.fit(X_train_class, y_train_class)

In [ ]:
# 最佳參數組合
print('最佳參數組合:')
pprint(grid_search.best_params_)
print()

# 最佳參數組合訓練的隨機森林模型
bestRfClassModel = grid_search.best_estimator_

# 模型預測
rfClass_train_pred = bestRfClassModel.predict(X_train_class)
rfClass_test_pred = bestRfClassModel.predict(X_test)

print('最佳參數組合隨機森林分類模型預測train的accuracy:', accuracy_score(y_train_class, rfClass_train_pred))
print('最佳參數組合隨機森林分類模型預測test的accuracy:', accuracy_score(y_test_class, rfClass_test_pred))

In [ ]:
# 繪製特徵解釋能力
importance = bestRfClassModel.feature_importances_
plotData = pd.DataFrame({'特徵名稱': X_train_raw.columns, '特徵重要程度': importance})
fig = px.bar(plotData, x='特徵名稱', y='特徵重要程度')
fig.show()

In [ ]:
# SHAP分析
explainer = shap.TreeExplainer(bestRfClassModel)
shap_values = explainer.shap_values(X_train_class)

In [ ]:
shap.summary_plot(shap_values[1], X_train_class, feature_names=featureChName)

# SVM(預測分類)

In [ ]:
from sklearn import svm

# 建立SVM分類模型
svcModel = svm.SVC()

# # 查看 SVC 默認參數值
# pprint(svcModel.get_params())

# 設定參數組合
paramGrid = {
    'C': [2**i for i in range(-10, 11, 2)],  # cost值
    'gamma': [2**i for i in range(-10, 11, 2)],  # gamma值
}

# 設定 Grid Search with Cross Validation
grid_search = GridSearchCV(estimator = svcModel, param_grid = paramGrid, 
                           cv = 5, n_jobs = -1, verbose = 2)

# 執行 Grid Search with Cross Validation
grid_search.fit(X_train_class, y_train_class)

In [ ]:
# 最佳參數組合
print('最佳參數組合:')
pprint(grid_search.best_params_)
print()

# 最佳參數組合訓練的SVC模型
bestSvcModel = grid_search.best_estimator_

# 模型預測
svc_train_pred = bestSvcModel.predict(X_train_class)
svc_test_pred = bestSvcModel.predict(X_test)

print('最佳參數組合SVC模型預測train的accuracy:', accuracy_score(y_train_class, svc_train_pred))
print('最佳參數組合SVC模型預測test的accuracy:', accuracy_score(y_test_class, svc_test_pred))

In [ ]:
# # SVM解釋器
# explainer = shap.KernelExplainer(bestSvcModel.predict, X_train_class)
# shap_values = explainer.shap_values(X_train_class)


# svm_explainer.fit(X_train_class)
# svm_explanation = svm_explainer.explain(X_train_class)
# shap.summary_plot(svm_explanation.shap_values[0], X_train_class, feature_names=featureChName)

# 整合預測結果

In [ ]:
# 整合預測值結果
predictRegResult = pd.DataFrame({'real': y_test_reg,
                                 'reg': reg_test_pred,
                                 'decisionTree': bestDecisionTreeReg_test_pred,
                                 'randomForest': rfReg_test_pred,
                                 'svm': svr_test_pred})
predictRegResult = pd.concat([testDataInfo, predictRegResult], axis=1)
predictRegResult

In [ ]:
# 整合預測分類結果
predictClassResult = pd.DataFrame({'real': y_test_class,
                                   'logistic': logistic_test_pred,
                                   'decisionTree': decisionTreeClass_test_pred,
                                   'randomForest': rfClass_test_pred,
                                   'svm': svc_test_pred})
predictClassResult = pd.concat([testDataInfo, predictClassResult], axis=1)
predictClassResult